# Install dependencies

In [2]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 6.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 5.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 6.5 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 kB 5.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.5/502.5 kB 4.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 1.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 9.2 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.1/68.1 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.9/256.9 kB 5.7 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 8.8 MB/s eta 0:00:00:00:0100:01


# Import libraries

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sqlite3 as sq3

# Data import and cleaning

In [24]:
#import homemade module for SQL interactions
from src.databases import SQLdb

#create a connection to obtain our database and store it in a pandas dataframe.
conn_match = SQLdb().open_connection("data/conduct_match.db")
df_match = pd.read_sql_query("SELECT * FROM match_data", conn)
SQLdb().close_connection(conn_match)

In [25]:
#first, look at a snapshot of the data
df_match.head(10)

,summonerName,matchId,gameVersion,queueId,championName,teamPosition,teamSide,gameDuration,kills,deaths,...,goldEarned,champExperience,visionScore,item0,item1,item2,item3,item5,item6,victory
0,Nayumi Shunni,NA1_4815992121,13.21.539.4823,420,Aatrox,TOP,100,1748,9,4,...,13844,16710,18,3161,6692,6694,3111,1037,3340,1
1,Bloalelye,NA1_4815992121,13.21.539.4823,420,LeeSin,JUNGLE,100,1748,6,3,...,13652,14674,28,3111,6692,3074,3071,1036,3340,1
2,ThęNihility,NA1_4815992121,13.21.539.4823,420,Swain,MIDDLE,100,1748,5,4,...,11071,13915,19,1056,3116,3105,3111,1033,3340,1
3,lardcow,NA1_4815992121,13.21.539.4823,420,Vayne,BOTTOM,100,1748,4,2,...,11481,12452,42,0,1055,3006,3087,3078,3340,1
4,Avô,NA1_4815992121,13.21.539.4823,420,Seraphine,UTILITY,100,1748,1,5,...,7690,10006,53,3222,3853,3158,2055,0,3364,1
5,Sakura Useless,NA1_4815992121,13.21.539.4823,420,MonkeyKing,TOP,200,1748,6,5,...,10826,13387,18,3071,6632,6609,3047,0,3340,0
6,nowin REE,NA1_4815992121,13.21.539.4823,420,Taliyah,JUNGLE,200,1748,5,8,...,10397,11112,33,4645,2421,6655,3158,1052,3364,0
7,T1 Mid,NA1_4815992121,13.21.539.4823,420,Malzahar,MIDDLE,200,1748,2,4,...,9547,13362,14,3040,3020,6653,1028,1052,3340,0
8,platyZ0A,NA1_4815992121,13.21.539.4823,420,Jhin,BOTTOM,200,1748,4,2,...,10930,11813,14,3033,3095,6671,3009,0,3363,0
9,I never jg lol,NA1_4815992121,13.21.539.4823,420,Leona,UTILITY,200,1748,1,6,...,6568,7637,69,3860,1011,2055,3190,3076,3364,0


We can see a representation of a single match spread across 10 rows, as evident by the same matchId. Each row represents the stats of an individual player in a game.

In [26]:
df_match.shape

(55280, 23)

With 10 players in each game, we are looking at a total of 5528 games in the database. Since the database schema arranges it so that the summonerName and matchId act as a primary key pair, we can be assured that all matches are unique.

In [27]:
#look at some general details of the data
df_match.describe()

,queueId,teamSide,gameDuration,kills,deaths,assists,damageDealt,damageTaken,goldEarned,champExperience,visionScore,item0,item1,item2,item3,item5,item6,victory
count,55280.0,55280.000000,55280.000000,55280.000000,55280.000000,55280.000000,55280.000000,55280.000000,55280.000000,55280.000000,55280.000000,55280.000000,55280.000000,55280.000000,55280.000000,55280.000000,55280.000000,55280.000000
mean,420.0,150.000000,1707.634949,5.770658,5.789978,7.972938,19295.346690,24852.280644,11220.040720,13102.993687,26.563622,3390.708412,3755.199258,3329.679758,3133.470351,1835.953672,3350.767855,0.500000
std,0.0,50.000452,441.734128,4.554377,3.212522,5.803355,11955.857317,12919.277501,3983.333542,4703.881226,21.659908,1863.340509,1838.958572,1713.111687,1741.207893,1794.209623,120.064936,0.500005
min,420.0,100.000000,101.000000,0.000000,0.000000,0.000000,0.000000,0.000000,500.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,420.0,100.000000,1489.000000,2.000000,3.000000,4.000000,10348.750000,15970.500000,8559.000000,10092.000000,13.000000,2065.000000,3020.000000,3006.000000,3006.000000,0.000000,3340.000000,0.000000
50%,420.0,150.000000,1739.000000,5.000000,6.000000,7.000000,17431.000000,23122.000000,11113.000000,13110.000000,20.000000,3111.000000,3134.000000,3085.000000,3074.000000,1054.000000,3363.000000,0.500000
75%,420.0,200.000000,1975.000000,8.000000,8.000000,11.000000,25771.000000,31895.000000,13774.250000,16154.000000,31.000000,3860.000000,4645.000000,3190.000000,3158.000000,3086.000000,3364.000000,1.000000
max,420.0,200.000000,3435.000000,34.000000,31.000000,47.000000,116818.000000,148952.000000,30388.000000,34376.000000,244.000000,8020.000000,8020.000000,8020.000000,8020.000000,8020.000000,3513.000000,1.000000


Let's make sure our data makes sense. 
- Queue ID being 420 for all games is correct since we specifically looked for Ranked Solo/Duo games with that same id number. 
- Team side should have an average of 150 since the "red team" is 100 and the "blue team" is 200, so we expect an equal amount of players from both teams.
- Game duration is in seconds. The shortest game was 101 seconds, which is around the time a team can remake a game if someone fails to connect. The longest game was almost an hour long, which I have personally experienced a handful of times.
- The average player will kill 5.77 other players, die 5.79 times, and get 7.97 assists (which is helping another player get a kill). The reason that deaths are very slightly larger than kills is that a player can die by other means than being killed by another player.
- Damage dealt/taken follows a similar pattern to kills/deaths, in that a player can get damaged in other ways besides other players. They can fight with non-player enemies that deal damage, such as monsters and enemy structures.
- Gold earned seems correct. Every player starts with 500 gold, and since a typical item costs around 3,000 gold to purchase, we expect each player to complete around 3-4 items before the game ends.
- Champion experience points is not something I am very knowledgeable about (it is a number to represent the player's level in the game). From a quick web search, I found that a player starts with 0 experience points and needs a total of 18360 experience points to reach level 18, the max level. The average experience points earned throughout a game is 13103, which is barely over level 15. This sounds logical.
- Vision score is an abstract metric which basically quantifies how many times you helped your team obtain information about the enemy team's location. The characters in the support role (or, in the words of the API, the "UTILITY" role) are typically the ones who can obtain the highest vision score. Since there is typically only one support per team, we see why there is such a big gap between the average of 26.56 vision score versus the highest vision score of 244.
- Each item has a unique identification number. While the mean doesn't have much meaning, one interesting fact we can glean from the data is that the item5 slot is hardly used compared to the others. Each item corresponds to their corresponding key on the keyboard, and the number 5 is far away from other keys used during a game. The reason item6 is used is because people usually change the keybind from 6 to another key.
- Lastly, victory is represented as 1 for a win and 0 for a loss. We expect and find that 0.5 is the average since one team will always win and the other team always loses in a match. Even if a game is remade due to a connection error, the team that voted for a remake is still deemed the "loser" despite none of the players obtaining any rewards or punishments for the game (except the person who failed to connect).

In [19]:
df_match.isnull().sum()

summonerName       0
matchId            0
gameVersion        0
queueId            0
championName       0
teamPosition       0
teamSide           0
gameDuration       0
kills              0
deaths             0
assists            0
damageDealt        0
damageTaken        0
goldEarned         0
champExperience    0
visionScore        0
item0              0
item1              0
item2              0
item3              0
item5              0
item6              0
victory            0
dtype: int64

Thanks to the API and structure of the database, it is not possible for any values to be null in the match database.

In [28]:
df_match.duplicated().sum()

0

We confirm that we have no duplicated rows in our data.
Now, we can move onto the player database.

In [30]:
#create a connection to obtain our database and store it in a pandas dataframe.
conn_player = SQLdb().open_connection("data/conduct_player.db")
df_player = pd.read_sql_query("SELECT * FROM player_data", conn_player)
SQLdb().close_connection(conn_player)

In [31]:
df_player.head(10)

,summonerName,region,puuid,leagueId,summonerId,tier,rank,leaguePoints,wins,losses,inactive
0,OnlyFullClear,na1,atKGMiKaVsk9i6NPfziLGEzPZISXgD9ruRWk6ESOlu-8rm...,5852e915-873e-41b5-bddf-80d78c4e2f9c,nqLB3jST2k5Qlqjh41jOQQo1gAj_oXgboZO19an7JS_5kB...,PLATINUM,II,0,15,25,0
1,nootherways,na1,G8xJV_OpnMwGUPmKpvtLjqBWabsqQoCDGYJlMqmCH-AUwI...,d0497be5-58c7-45c6-8c58-1c3cd1d2db38,LKCThiSSoJxP9LgmDZedJKEdckckC13l2gXorol1Cv_tH4o,EMERALD,III,21,247,223,0
2,ßººß,na1,7zBVk6HEvwUbW6C8VnSvqYxquOPkYM2QkaFwWLk_1v5N6H...,df960352-50f9-4306-812a-6e827aa69a68,Zq5fbRPHE8mkT27Zbh0zEHIgI915cbJh-vcEqpoku09bAjM,EMERALD,III,31,135,136,0
3,Brimreach,na1,VJfGX2-hgaNckuoFN6e0lT2tuc0nqm-5lxqbGxI0_eLgFH...,d4caf6a7-b3c7-412c-b05b-ace556b5a728,qMG5HQKHKvHiZXn0gi-0MmBUSNCO4IgCsM5A_0iMo1kNV6...,EMERALD,IV,0,276,272,0
4,ßäłłš,na1,RxF1jmSLJIZYbAmkga8kyEgdtipsIDaoKvn8d-cuUGfcgr...,3d37f5dd-dd80-42f6-b998-fc1bed587c13,yP9IWIyq_-G1j-mZM_vWHEXwwu0KoswpA7d_NpinxIuUGE...,PLATINUM,IV,75,13,16,0
5,Savage Prodigy,na1,O-uLtymTa11yknVtgZmqXLelb8iH_jTEtwCTxOP1ESFuih...,c7ea2b2e-db95-4ec3-8eab-7e6a41001603,jFzQ11_7B0sui_zojWDUwN6j7CbWUb-FudWadhcCzFGGhV...,EMERALD,III,0,191,191,0
6,JunaLye,na1,xKGg-884LrNHtRcnhK0-IkYCIVWyT_A1sgDCxnLko8ncuK...,a2e185b2-df23-4a51-9074-dab2053a31f3,AjF9-7j_oiLHdbkcr-oY2T5VFaJyVhI0hpnJ0Nzu5okzqHYP,PLATINUM,IV,23,67,75,0
7,FootGooner27,na1,R09G3-h6oxmX6EDAJG_gNus6S8t8OhLxQkBgJrmsLZy0qp...,1479063a-fddd-4edd-823c-a3fa8c473151,SxP0bxj-7n3T83g52B02G_kBiK1DX-DqZD9d5yU-PSu8haM,EMERALD,III,75,34,36,0
8,Vyhaega,na1,82mSnvDzAsa3zD75dNPyHuGbNsJgwUagDmLMcEGddo5Ydv...,591385f4-992e-47d6-b57d-f211ff5cf9f3,Yo_Co76yiAyp8eMCNJtVC_fiOh0nb2UkSSFOzD-NXoecho...,EMERALD,IV,50,52,39,0
9,HeisenFaker,na1,zITN1XtTfPmWxoNlf7KUSPbul637mvFBmYOaRaaW16TVP-...,9b99f93a-99c3-4297-be7a-ca804c120274,2MyWR3rahPc77pWVisL0OW42weCety7nD9Bzhnh1tdRbrc8,EMERALD,III,28,71,82,0


Each row represents the information about a specific player, including their ID numbers and ranked title, as well as their win-loss record.

In [32]:
df_player.shape

(36497, 11)

We should have 36497 unique players in our database. We have 55280 matches, and we expect a roughly concentrated amount of players due to the data scraping method employed (analyzing a handful of matches of a single player then choosing a random match from said player and choosing a random player from said match).

In [34]:
df_player.describe()

,leaguePoints,wins,losses,inactive
count,36497.000000,36497.000000,36497.000000,36497.000000
mean,33.827027,95.066636,90.981286,0.011645
std,29.471124,85.319834,84.414741,0.107282
min,0.000000,0.000000,0.000000,0.000000
25%,3.000000,33.000000,31.000000,0.000000
50%,28.000000,72.000000,67.000000,0.000000
75%,56.000000,131.000000,126.000000,0.000000
max,499.000000,982.000000,971.000000,1.000000


- League points (LP) are used in the ranked system for League of Legends. Each tier (Bronze, Silver, Gold, etc) has an associated rank (IV, III, II, I), and you need 100 LP to move up a rank. Starting from Master tier, there are no longer any ranks and their league points become uncapped, which explains the max value of 499 LP.
- A player in our database won 5 more times than lost on average. There could be two reasons for this: 1. a player who plays a lot is more likely to be scraped by our program. A veteran player might be more favored to win than a player who plays less, or 2. players occasionally create new accounts to play in a lower ranked division, which is called "smurfing". These players are able to beat their opponents with ease due to the skill imbalance.
- Inactive is a boolean for inactivity. We see that around 1% of players in this database are inactive. 